In [1]:
"""
Synthetic 3D Time-Series Generator
--------------------------------------
For the Nature Reviews Methods Primer

This script generates a synthetic 3D time-series of 2D images, featuring oscillating circular regions 
and propagating waves. It includes various features such as transient signals, quasi-periodic components, 
and a fluting mode to simulate complex dynamics, plus noise.

The final time series is saved in FITS format for further analysis.
"""
import numpy as np # type: ignore
from astropy.io import fits # type: ignore
from tqdm import tqdm # type: ignore    

# Parameters for synthetic 2D image creation
synthetic_image = [130., 50., 100., 2.]
image_size = int(synthetic_image[0])
num_oscillation_regions = int(synthetic_image[1])
largest_radius = int(synthetic_image[2])
radius_decrement = int(synthetic_image[3])

# Image center
center_x = image_size / 2
center_y = image_size / 2

# Oscillation region info
oscillation_region_info = np.zeros((3, num_oscillation_regions))
for region_index in range(num_oscillation_regions):
    oscillation_region_info[0, region_index] = center_x
    oscillation_region_info[1, region_index] = center_y
    oscillation_region_info[2, region_index] = largest_radius - radius_decrement * region_index

# Wave parameters
num_waves = 10
intensity_amplitudes = [1.60, 1.80, 1.90, 1.70, 1.20, 2.00, 2.40, 1.20, 1.80, 2.20]
intensity_frequencies = [0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55]
phase_shifts = [0, 0.5, 1.0, 1.5, 2.0, 0.25, 0.75, 1.25, 1.75, 2.25] # in radians

# Time series parameters
num_frames = 200.
duration = 99.5
num_frames = int(num_frames)  # Convert to integer
time = np.linspace(0, duration, num_frames)

# Transient signal parameters
polynomial_coefficients = [0.01, -0.02, 0.03]

# Transverse oscillation parameters
sway_amplitude_x = 14.0
sway_amplitude_y = 10.0
sway_frequency_x = 0.25
sway_frequency_y = 0.25

# Fluting mode parameters
fluting_amplitude = 14.0
fluting_frequency = 0.50
fluting_wavenumber = 3

# Quasi-periodic signal parameters
quasi_amplitude = 5.2
quasi_frequency = 0.55
quasi_phase_shift = 0.5 # in radians

# Initialize time series (with and without noise)
time_series = np.zeros((image_size, image_size, num_frames))
time_series_no_noise = np.zeros((image_size, image_size, num_frames))

# Generate the 3D time series
for k in tqdm(range(num_frames), desc="Generating Frames", unit="frame"):
    transient_signal = np.polyval(polynomial_coefficients[::-1], k / num_frames)
    sway_x = sway_amplitude_x * np.sin(2 * np.pi * sway_frequency_x * time[k])
    sway_y = sway_amplitude_y * np.sin(2 * np.pi * sway_frequency_y * time[k])
    quasi_periodic_signal = quasi_amplitude * np.sin(2 * np.pi * quasi_frequency * time[k] + quasi_phase_shift)

    if k == 3:
        angleout = np.zeros((image_size, image_size))

    for i in range(image_size):
        for j in range(image_size):
            intensity = 0.0
            dx = j - center_x + sway_x
            dy = i - center_y + sway_y

            angle = np.arctan2(dy, dx).astype(np.float32)
            if angle < 0:
                angle += 2 * np.pi
            angle = np.round(angle, 5)
            
            distance_to_center = np.sqrt(dx**2 + dy**2)

            if k == 3:
                angleout[i,j] = dy

            # rotation_offset = np.pi / 6  # Example offset (tune as needed)
            # angle = angle - rotation_offset

            for region_index in range(num_oscillation_regions):
                region_radius = oscillation_region_info[2, region_index]
                fluting_effect = 0.0
                if num_oscillation_regions / 2 - 2 <= region_index <= num_oscillation_regions / 2:
                    fluting_effect = fluting_amplitude * np.sin(fluting_wavenumber * angle) * np.sin(2 * np.pi * fluting_frequency * time[k])

                local_frequency = intensity_frequencies[region_index % num_waves]

                for wave in range(num_waves):
                    wave_amplitude = intensity_amplitudes[wave]
                    wave_phase_propagating = 2 * np.pi * local_frequency * time[k]
                    wave_shape = np.sin(2 * np.pi * local_frequency * distance_to_center / region_radius + phase_shifts[wave] + wave_phase_propagating)
                    intensity += wave_amplitude * wave_shape

                intensity += fluting_effect

            time_series_no_noise[i, j, k] = intensity * (1 + transient_signal) + quasi_periodic_signal
            time_series[i, j, k] = intensity * (1 + transient_signal) + quasi_periodic_signal + (np.random.uniform(-0.5, 0.5) * 0.1)

# Transpose time series to match IDL convention ([t, x, y])
time_series_no_noise = np.transpose(time_series_no_noise, (2, 0, 1))
time_series = np.transpose(time_series, (2, 0, 1))

# Save the 3D time series to FITS files
fits_file = "Synthetic_Data/NRMP_signal_3D.fits"
hdu_signal = fits.PrimaryHDU(time_series)
hdu_signal.header["TIME"] = "SEE EXT=1 OF THE FITS FILE"
hdu_signal.writeto(fits_file, overwrite=True)

hdu_time = fits.ImageHDU(time)
hdul = fits.HDUList([hdu_signal, hdu_time])
hdul.writeto(fits_file, overwrite=True)

Generating Frames: 100%|██████████| 200/200 [28:11<00:00,  8.46s/frame]
